In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("../data/checking-logs.sqlite")
cur = con.cursor()

In [7]:
import pandas as pd
import sqlite3

# Create connection
conn = sqlite3.connect('../data/checking-logs.sqlite')

avg_first_view = pd.read_sql("""
    SELECT AVG(CAST(strftime('%s', first_view_ts) AS INT)) as avg_timestamp 
    FROM test
""", conn).iloc[0,0]

test_results = pd.read_sql("""
    WITH time_periods AS (
        SELECT 
            uid,
            'before' AS time,
            AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines)/3600.0) AS avg_diff
        FROM test t
        JOIN deadlines d ON t.labname = d.labs
        WHERE t.labname != 'project1'
        AND CAST(strftime('%s', first_commit_ts) AS INT) < CAST(strftime('%s', first_view_ts) AS INT)
        GROUP BY uid
        
        UNION ALL
        
        SELECT 
            uid,
            'after' AS time,
            AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines)/3600.0) AS avg_diff
        FROM test t
        JOIN deadlines d ON t.labname = d.labs
        WHERE t.labname != 'project1'
        AND CAST(strftime('%s', first_commit_ts) AS INT) >= CAST(strftime('%s', first_view_ts) AS INT)
        GROUP BY uid
    ),
    users_with_both AS (
        SELECT uid
        FROM time_periods
        GROUP BY uid
        HAVING COUNT(DISTINCT time) = 2
    )
    SELECT 
        time,
        AVG(avg_diff) AS avg_diff
    FROM time_periods
    WHERE uid IN (SELECT uid FROM users_with_both)
    GROUP BY time
    ORDER BY time DESC
""", conn)
test_results

,time,avg_diff
0,before,-66.679583
1,after,-100.178181


In [8]:
control_results = pd.read_sql(f"""
    WITH time_periods AS (
        SELECT 
            uid,
            'before' AS time,
            AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines)/3600.0) AS avg_diff
        FROM control c
        JOIN deadlines d ON c.labname = d.labs
        WHERE c.labname != 'project1'
        AND CAST(strftime('%s', first_commit_ts) AS INT) < {avg_first_view}
        GROUP BY uid
        
        UNION ALL
        
        SELECT 
            uid,
            'after' AS time,
            AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines)/3600.0) AS avg_diff
        FROM control c
        JOIN deadlines d ON c.labname = d.labs
        WHERE c.labname != 'project1'
        AND CAST(strftime('%s', first_commit_ts) AS INT) >= {avg_first_view}
        GROUP BY uid
    ),
    users_with_both AS (
        SELECT uid
        FROM time_periods
        GROUP BY uid
        HAVING COUNT(DISTINCT time) = 2
    )
    SELECT 
        time,
        AVG(avg_diff) AS avg_diff
    FROM time_periods
    WHERE uid IN (SELECT uid FROM users_with_both)
    GROUP BY time
    ORDER BY time DESC
""", conn)
control_results

,time,avg_diff
0,before,-98.467852
1,after,-99.803549


In [5]:
con.close()

# Результаты тестовой группы

До просмотра Newsfeed: студенты сдавали работы в среднем на 66.7 часов раньше дедлайна

После просмотра Newsfeed: студенты сдавали работы в среднем на 100.2 часов раньше дедлайна

Разница: -33.5 часов (увеличение заблаговременности сдачи на 50%)

# Результаты контрольной группы

До условного времени: -98.5 часов

После условного времени: -99.8 часов

Разница: -1.3 часа (незначительное изменение)

# Эффект

В тестовой группе после просмотра Newsfeed студенты начали сдавать работы значительно раньше (разница 33.5 часов)

В контрольной группе нет существенных изменений в поведении (разница всего 1.3 часа)